This was my work for **HackerEarth: Employee Burnout Hackathon**  
I only discovered it as it was about to end, so couldnt work on it a much  
A few plots for eda, followed by model building and creation of submission file

To read more about the problem: https://www.hackerearth.com/problem/machine-learning/predict-the-employee-burn-out-rate-7-6340b4e3/

### Problem Statement
World Mental Health Day is celebrated on October 10 each year. The objective of this day is to raise an awareness about mental health issues around the world and mobilise efforts in support of mental health. According to an anonymous survey, about 450 million people live with mental disorders that can be one of the primary causes of poor health and disability worldwide.  

You are a Machine Learning engineer in a company. You are given a task to understand and observe the mental health of all the employees in your company. Therefore, you are required to predict the burn out rate of employees based on the provided features thus helping the company to take appropriate measures for their employees.

#### Importing necessary modules and reading the data in as pandas dataframe

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('../input/are-your-employees-burning-out/train.csv', parse_dates=['Date of Joining'])
test_df = pd.read_csv('../input/are-your-employees-burning-out/test.csv', parse_dates=['Date of Joining'])
sample_submission = pd.read_csv('../input/are-your-employees-burning-out/sample_submission.csv')

In [ ]:
df.head()

In [ ]:
test_df.head()

#### Check for missing values and drop rows with any

In [ ]:
df.isna().sum()

In [ ]:
test_df.isna().sum()

In [ ]:
## Missing values in training data for 3 columns
## Currently drop all rows with any na values
df = df.dropna(how='any')

In [ ]:
df.dtypes

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
sns.countplot(df['Gender'], ax = axs[0])
axs[0].set_title('Training Data')
axs[0].set_xlabel('Gender')
axs[0].set_ylabel('Count')

sns.countplot(test_df['Gender'], ax = axs[1])
axs[1].set_title('Testing Data')
axs[1].set_xlabel('Gender')
axs[1].set_ylabel('Count')

In [ ]:
## Gender encoding 0 for male and 1 for female
df['Gender'] = df['Gender'].apply(lambda x: 0 if x=='Male' else 1)
test_df['Gender'] = test_df['Gender'].apply(lambda x: 0 if x=='Male' else 1)

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
sns.countplot(df['Company Type'], ax = axs[0])
axs[0].set_title('Training Data')
axs[0].set_xlabel('Company Type')
axs[0].set_ylabel('Count')

sns.countplot(test_df['Company Type'], ax = axs[1])
axs[1].set_title('Testing Data')
axs[1].set_xlabel('Company Type')
axs[1].set_ylabel('Count')

In [ ]:
## Company Type is binary categorical

df['Company Type'] = df['Company Type'].apply(lambda x: 0 if x == 'Service' else 1)
test_df['Company Type'] = test_df['Company Type'].apply(lambda x: 0 if x == 'Service' else 1)

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,5))
sns.countplot(df['WFH Setup Available'], ax = axs[0])
axs[0].set_title('Training Data')
axs[0].set_xlabel('WFH Setup')
axs[0].set_ylabel('Count')

sns.countplot(test_df['WFH Setup Available'], ax = axs[1])
axs[1].set_title('Testing Data')
axs[1].set_xlabel('WFH Setup')
axs[1].set_ylabel('Count')



## WFH Encoding Binary

df['WFH Setup Available'] = df['WFH Setup Available'].apply(lambda x: 0 if x == 'No' else 1)
test_df['WFH Setup Available'] = test_df['WFH Setup Available'].apply(lambda x: 0 if x == 'No' else 1)

In [ ]:
sns.countplot(df['Designation'], color='green', alpha = 0.8)
plt.title('Designation Countplot')


## Convert Designations to integers

df['Designation'] = df['Designation'].astype('int8')
test_df['Designation'] = test_df['Designation'].astype('int8')

In [ ]:
sns.countplot(df['Resource Allocation'], color='green', alpha = 0.8)
plt.title('Resource Allocation Countplot')

## Convert Designations to integers

df['Resource Allocation'] = df['Resource Allocation'].astype('int8')
test_df['Resource Allocation'] = test_df['Resource Allocation'].astype('int8')

In [ ]:
sns.distplot(df['Mental Fatigue Score'])
df['Mental Fatigue Score'].describe()

In [ ]:
sns.distplot(df['Burn Rate'])
df['Burn Rate'].describe()

In [ ]:
## Check range of Date variable
df['Date of Joining'].max() - df['Date of Joining'].min() ## Data over timeperiod of 1year

In [ ]:
### Moving 'Date of Joining' start from zero and Min max scale manually

df['Date of Joining'] = df['Date of Joining'].apply(lambda x: x- df['Date of Joining'].min())
df['Date of Joining'] = df['Date of Joining'].apply(lambda x: x.days)
df['Date of Joining'] = df['Date of Joining'].apply(lambda x: x/365)

test_df['Date of Joining'] = test_df['Date of Joining'].apply(lambda x: x- test_df['Date of Joining'].min())
test_df['Date of Joining'] = test_df['Date of Joining'].apply(lambda x: x.days)
test_df['Date of Joining'] = test_df['Date of Joining'].apply(lambda x: x/365)


#### Plotting correlation heatmap

In [ ]:
mask = np.triu(np.ones_like(df.corr()))
plt.figure(figsize=(10,10))
heatmap = sns.heatmap(df.corr(), mask=mask,
                      vmin=-1, vmax=1, annot=True, cmap='BrBG')

#### Reformatting data for training model

In [ ]:
X = df.drop(['Burn Rate','Employee ID'], axis='columns')
y = df['Burn Rate']

t = test_df.drop(['Employee ID'], axis='columns')

In [ ]:
t.head()

#### Model building, training and making predictions

In [ ]:
pf = PolynomialFeatures(2) ## sklearn class for making polynomial features of input variables
X = pf.fit_transform(X.drop('Gender', axis=1))
t = pf.transform(t.drop('Gender', axis=1))
lr = LinearRegression() ## Model Build
lr.fit(X,y)
lr.score(X,y)

In [ ]:
predictions = lr.predict(t)

#### Creating a submission file

In [ ]:
def create_submission(file_name, predictions):
    pred = pd.DataFrame(predictions, index=test_df['Employee ID'], columns=['Burn Rate'])
    pred.reset_index(inplace=True)
    pred.to_csv(file_name + '.csv', index=False)

In [ ]:
create_submission('PolyRegression deg2', predictions)
pd.read_csv('PolyRegression deg2.csv').head()